# Knapsack Problem
Given a set of items with assigned values and weights. Fill a container such that the value in the container is maximized while still being under the weight limit

Solved with BQM

In [4]:
# import necessary things
import time
import dimod

import numpy as np
import numpy.random as random

from pyqubo import Binary, Constraint

from dwave.system import DWaveSampler, AutoEmbeddingComposite

## Generate set of items and weights and set weight limit

In [54]:
# nummber of items
n = 50

# weight limit
W = 30

# generate arrays for values and weights
random.seed(50)
values = random.randint(1,10,n)
weights = random.randint(1,10,n)

print('values: ', values)
print('weights:', weights)

values:  [1 1 2 5 7 6 7 7 6 3 8 5 4 7 5 2 6 1 7 4 3 4 4 4 3 1 4 3 1 4 1 1 8 4 9 8 5
 5 1 1 4 4 2 5 6 8 1 4 6 7]
weights: [2 5 5 5 6 5 7 4 1 6 9 4 7 3 9 9 6 5 8 9 5 5 3 2 9 8 2 6 9 4 4 6 4 7 9 7 1
 9 3 2 2 7 6 6 7 5 5 1 1 8]


## Build BQM

In [88]:
# build bqm
P = 10

# build variables
# binary variable x_i for if item i is packed or not
x = [Binary(f'{i}') for i in range(n)]

# value objective
H = 0
for i in range(len(x)):
    H -= values[i]*x[i]

# slack vars
s = [Binary(f's{i}') for i in range(4)]
sw = [2**i for i in range(4)]

# weight constraint
H += P*Constraint((W - np.dot(weights,x) - np.dot(sw,s))**2, label='weight constraint')

model = H.compile()
Q = model.to_bqm()

### Check problem requirements (time, vars, constraints)

In [89]:
sampler = DWaveSampler()
print("num vars:",len(Q.variables))

num vars: 54


## Sample

In [93]:
start = time.time()
# run QPU solver
sampler = AutoEmbeddingComposite(DWaveSampler())
response = sampler.sample(Q, num_reads=1000, label='BQM Knapsack')
elapsed = time.time() - start
print("Solved in %.2f seconds" % elapsed)
print(response.first.sample)

Solved in 14.74 seconds
{'0': 1, '1': 0, '10': 0, '11': 1, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '2': 0, '20': 0, '21': 0, '22': 1, '23': 0, '24': 0, '25': 0, '26': 0, '27': 1, '28': 0, '29': 0, '3': 0, '30': 0, '31': 0, '32': 1, '33': 0, '34': 0, '35': 0, '36': 1, '37': 0, '38': 1, '39': 0, '4': 0, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47': 0, '48': 0, '49': 0, '5': 1, '6': 0, '7': 0, '8': 1, '9': 0, 's0': 1, 's1': 0, 's2': 0, 's3': 0}


In [94]:
soln_vals = np.zeros(n)
for i in response.first.sample:
    try:
        soln_vals[int(i)] = response.first.sample[i]
    except:
        continue
print(soln_vals)
value = 0
weight = 0
for i in range(len(x)):
    value += soln_vals[i]*values[i]
    weight += soln_vals[i]*weights[i]
print('value:',value)
print('weight:',weight)

[1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
value: 39.0
weight: 29.0


In [95]:
dec = model.decode_sample(response.first.sample, vartype='BINARY')
print(dec.constraints())

{'weight constraint': (True, 0.0)}
